In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

us_ed_df = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

1. Determine all the variable types and find the fraction of the missing values for each variable.

In [3]:
us_ed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

In [4]:
us_ed_df.isna().mean()

PRIMARY_KEY                     0.000000
STATE                           0.000000
YEAR                            0.000000
ENROLL                          0.176273
TOTAL_REVENUE                   0.142091
FEDERAL_REVENUE                 0.142091
STATE_REVENUE                   0.142091
LOCAL_REVENUE                   0.142091
TOTAL_EXPENDITURE               0.142091
INSTRUCTION_EXPENDITURE         0.142091
SUPPORT_SERVICES_EXPENDITURE    0.142091
OTHER_EXPENDITURE               0.176273
CAPITAL_OUTLAY_EXPENDITURE      0.142091
GRADES_PK_G                     0.115952
GRADES_KG_G                     0.088472
GRADES_4_G                      0.087802
GRADES_8_G                      0.087802
GRADES_12_G                     0.087802
GRADES_1_8_G                    0.087802
GRADES_9_12_G                   0.087802
GRADES_ALL_G                    0.115952
AVG_MATH_4_SCORE                0.640751
AVG_MATH_8_SCORE                0.643432
AVG_READING_4_SCORE             0.642761
AVG_READING_8_SC

Question 2. Notice that the data has a time dimension (year). For this assignment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?

Answer: Because the time dimension has been removed, we cannot interpolate the missing values.  The reading and math scores are missing at a very high percentage, so those columns need to be excluded from our analysis.  Grades are missing at a low percentage, so we might just imputate those with a median value.  Expenditure and revenue columns are missing at a higher percentage, so multi-imputation might be the best approach.  However, because we haven't covered how to do that, let's just replace missing values with the mean for fun!

In [5]:
us_ed_df2 = us_ed_df.copy()

cols = [col for col in us_ed_df2.columns if col not in ['PRIMARY_KEY', 'STATE']]

for col in cols:
    us_ed_df2.loc[:,col] = us_ed_df2.loc[:,col].fillna(us_ed_df2.loc[:,col].mean())
    

In [6]:
#Checking to make sure values were filled with mean
us_ed_df2.isna().mean()

PRIMARY_KEY                     0.0
STATE                           0.0
YEAR                            0.0
ENROLL                          0.0
TOTAL_REVENUE                   0.0
FEDERAL_REVENUE                 0.0
STATE_REVENUE                   0.0
LOCAL_REVENUE                   0.0
TOTAL_EXPENDITURE               0.0
INSTRUCTION_EXPENDITURE         0.0
SUPPORT_SERVICES_EXPENDITURE    0.0
OTHER_EXPENDITURE               0.0
CAPITAL_OUTLAY_EXPENDITURE      0.0
GRADES_PK_G                     0.0
GRADES_KG_G                     0.0
GRADES_4_G                      0.0
GRADES_8_G                      0.0
GRADES_12_G                     0.0
GRADES_1_8_G                    0.0
GRADES_9_12_G                   0.0
GRADES_ALL_G                    0.0
AVG_MATH_4_SCORE                0.0
AVG_MATH_8_SCORE                0.0
AVG_READING_4_SCORE             0.0
AVG_READING_8_SCORE             0.0
dtype: float64

3. Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using only the observations for that specific year.

In [9]:
us_ed_df3 = us_ed_df.copy()

cols = [col for col in us_ed_df.columns if col not in ['PRIMARY_KEY', 'STATE']]
years = us_ed_df3['YEAR'].unique()

for col in cols:
    for year in years:
        us_ed_df3.loc[us_ed_df3['YEAR'] == year,col] = us_ed_df3.loc[us_ed_df3['YEAR'] == year,col].fillna(us_ed_df3[us_ed_df3['YEAR'] == year][col].mean()) 
        #education3_df.loc[education3_df["YEAR"] == year, col] = education3_df.loc[education3_df["YEAR"] == year, col].fillna(education3_df[education3_df["YEAR"] == year][col].mean())


In [10]:
us_ed_df3.isna().mean()

PRIMARY_KEY                     0.000000
STATE                           0.000000
YEAR                            0.000000
ENROLL                          0.071716
TOTAL_REVENUE                   0.034182
FEDERAL_REVENUE                 0.034182
STATE_REVENUE                   0.034182
LOCAL_REVENUE                   0.034182
TOTAL_EXPENDITURE               0.034182
INSTRUCTION_EXPENDITURE         0.034182
SUPPORT_SERVICES_EXPENDITURE    0.034182
OTHER_EXPENDITURE               0.071716
CAPITAL_OUTLAY_EXPENDITURE      0.034182
GRADES_PK_G                     0.068365
GRADES_KG_G                     0.068365
GRADES_4_G                      0.068365
GRADES_8_G                      0.068365
GRADES_12_G                     0.068365
GRADES_1_8_G                    0.068365
GRADES_9_12_G                   0.068365
GRADES_ALL_G                    0.068365
AVG_MATH_4_SCORE                0.576408
AVG_MATH_8_SCORE                0.576408
AVG_READING_4_SCORE             0.576408
AVG_READING_8_SC

4. This time, fill in the missing values using interpolation (extrapolation).

In [12]:
us_ed_df4 = us_ed_df.copy()
cols = [col for col in us_ed_df.columns if col not in ['PRIMARY_KEY', 'STATE']]

for col in cols:
    us_ed_df4.loc[:,col] = us_ed_df4.loc[:,col].interpolate(limit_direction='both')

In [13]:
us_ed_df4.isna().mean()

PRIMARY_KEY                     0.0
STATE                           0.0
YEAR                            0.0
ENROLL                          0.0
TOTAL_REVENUE                   0.0
FEDERAL_REVENUE                 0.0
STATE_REVENUE                   0.0
LOCAL_REVENUE                   0.0
TOTAL_EXPENDITURE               0.0
INSTRUCTION_EXPENDITURE         0.0
SUPPORT_SERVICES_EXPENDITURE    0.0
OTHER_EXPENDITURE               0.0
CAPITAL_OUTLAY_EXPENDITURE      0.0
GRADES_PK_G                     0.0
GRADES_KG_G                     0.0
GRADES_4_G                      0.0
GRADES_8_G                      0.0
GRADES_12_G                     0.0
GRADES_1_8_G                    0.0
GRADES_9_12_G                   0.0
GRADES_ALL_G                    0.0
AVG_MATH_4_SCORE                0.0
AVG_MATH_8_SCORE                0.0
AVG_READING_4_SCORE             0.0
AVG_READING_8_SCORE             0.0
dtype: float64

5. Compare your results for the 2nd, 3rd, and 4th questions. Do you find any meaningful differences?

In [18]:
cols = [col for col in us_ed_df.columns if col not in ['PRIMARY_KEY', 'STATE','YEAR']]

for col in cols:
    temp_df = pd.concat([us_ed_df[col],us_ed_df2[col],us_ed_df3[col],us_ed_df4[col]],axis=1)
    print(temp_df.median())
    

ENROLL    648313.0
ENROLL    820414.0
ENROLL    737401.0
ENROLL    737393.5
dtype: float64
TOTAL_REVENUE    5079546.0
TOTAL_REVENUE    6359310.5
TOTAL_REVENUE    5778507.0
TOTAL_REVENUE    6351759.5
dtype: float64
FEDERAL_REVENUE    403376.5
FEDERAL_REVENUE    516741.0
FEDERAL_REVENUE    446180.0
FEDERAL_REVENUE    516519.0
dtype: float64
STATE_REVENUE    2537073.5
STATE_REVENUE    3127639.0
STATE_REVENUE    2857497.0
STATE_REVENUE    3119414.5
dtype: float64
LOCAL_REVENUE    2055780.5
LOCAL_REVENUE    2697257.0
LOCAL_REVENUE    2398344.0
LOCAL_REVENUE    2650904.0
dtype: float64
TOTAL_EXPENDITURE    5234505.5
TOTAL_EXPENDITURE    6520223.5
TOTAL_EXPENDITURE    5858385.0
TOTAL_EXPENDITURE    6499891.0
dtype: float64
INSTRUCTION_EXPENDITURE    2657452.0
INSTRUCTION_EXPENDITURE    3358141.5
INSTRUCTION_EXPENDITURE    3033016.0
INSTRUCTION_EXPENDITURE    3318777.5
dtype: float64
SUPPORT_SERVICES_EXPENDITURE    1525405.5
SUPPORT_SERVICES_EXPENDITURE    1913843.5
SUPPORT_SERVICES_EXPENDITUR

Yes, there do seem to be significant differences between the methods.